In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import time
import json
pd.set_option('display.max_columns', None)

In [5]:
schedule = pd.read_csv('sportrader_schedule.csv')
schedule

,Unnamed: 0,Game ID,Home Team,Away Team,Home Score,Away Score,Stadium
0,0,06d50f40-10ca-4918-bda7-7df773a771f0,Cleveland Browns,Tennessee Titans,13,43,FirstEnergy Stadium
1,1,26c758eb-ebf1-46aa-9fa8-6934a8a74b23,Miami Dolphins,Baltimore Ravens,10,59,Hard Rock Stadium
2,2,31c88647-6e15-47c5-b196-e03a7991a75b,Oakland Raiders,Denver Broncos,24,16,RingCentral Coliseum
3,3,3242f9ee-37c7-4508-aac9-6fb98259d673,New Orleans Saints,Houston Texans,30,28,Mercedes-Benz Superdome
4,4,388f432f-f522-4552-9d81-55d64e804066,Seattle Seahawks,Cincinnati Bengals,21,20,CenturyLink Field
...,...,...,...,...,...,...,...
251,251,b3162a63-cc56-4d65-b46d-3fa127aac2ac,Buffalo Bills,New York Jets,6,13,New Era Field
252,252,b40a6567-364f-4e1d-91b2-695cbd30a683,New York Giants,Philadelphia Eagles,17,34,MetLife Stadium
253,253,c48c7db4-27ca-49b8-8bdd-068573226616,Carolina Panthers,New Orleans Saints,10,42,Bank of America Stadium
254,254,f73a5d00-a948-4cfe-9b96-3d3e60db8006,Baltimore Ravens,Pittsburgh Steelers,28,10,M&T Bank Stadium


In [16]:
#Running the function below with your api_key and the game_id you want to scrape will return a DataFrame of relatively flattened json
#Use a game_id from the schedule to check out the result!
#Flexible to be used for earlier seasons as well.
def game_scraper(api_key,game_id):
    import http.client
    import pandas as pd
    import json
    from pandas.io.json import json_normalize
    from pprint import pprint
    #Pull PBP JSON From Sportradar 
    conn = http.client.HTTPSConnection("api.sportradar.us")
    conn.request("GET", f"/nfl/official/trial/v5/en/games/{game_id}/pbp.json?api_key={api_key}")
    res = conn.getresponse()
    data_frame = res.read()
    
    #load Json into a df
    df_2 = json.loads(data_frame)
    
    #The PBP Data is nested inside of the Periods, lets append it to a list so that we can access it easier
    listy = []
    for i in range(len(df_2['periods'])):
        woooh = json.dumps(df_2['periods'][i]['pbp'])
        listy.append(json.loads(woooh))
    #Creating a dictionary out of the Plays in order to create a nice DataFrame. 
    #Using PlayCounter in order to add to Dictionary and as an index to merge future data on
    dictionary = {}
    play_counter = 0
    for i in range(len(listy)):
        for x in range(len(listy[i])):
            try:
                nexty = listy[i][x]['events']
                for z in range(len(nexty)):
                    play_counter = play_counter + 1
                    dictionary[f'play #{play_counter}'] = nexty[z]
            except KeyError:
                continue
    # Create DF from Dictionary so that we can begin making the json pretty
    hmm = pd.DataFrame.from_dict(dictionary).transpose()
    hmm = hmm.reset_index()
    #creating columns
    hmm['yardline'] = 'creation'
    hmm['down'] = 'creation'
    hmm['yards to go'] = 'creation'
    hmm['possession'] = 'creation'
    #Once DF is created, there are more Json objects nested inside. We do the below to extract Data we can use
    for yard in range(len(hmm)):
        try:
            hmm['yardline'][yard] = hmm['start_situation'][yard]['location']['yardline']
            hmm['down'][yard] = hmm['start_situation'][yard]['down']
            hmm['yards to go'][yard] = hmm['start_situation'][yard]['yfd']
            hmm['possession'][yard] = hmm['start_situation'][yard]['possession']['name']
        except TypeError:
            hmm['yardline'][yard] = "NaN"
            hmm['down'][yard] = "NaN"
            hmm['possession'][yard] = "NaN"
            hmm['yards to go'][yard] = "NaN"
    #The Most Important details are in the Statistics and Details nested Json inside the PBP. Similar process to the PBP flattening above        
    stats = {}
    details = {}
    for i in range(len(hmm['statistics'])):
        try:
            for x in range(len(hmm['statistics'][i])):
                try:
                    stats[f'play #{i+1} , stat {x}'] = hmm['statistics'][i][x]
                except TypeError:
                    continue
            for y in range(len(hmm['details'][i])):
                try:
                    details[f'play #{i+1} , detail {y}'] = hmm['details'][i][y]
                except TypeError:
                    continue
        except TypeError:
            continue
    #The Stats json is broken up into different categories (usually one or two for offense,one for defense.)
    #We keep note of the stat_type so that we can filter the data for the stats we want
    #Create DataFrames
    statsy = pd.DataFrame.from_dict(stats).transpose()
    statsy = statsy.reset_index()
    deets = pd.DataFrame.from_dict(details).transpose()
    deets = deets.reset_index()
    #Create Indexes in order to Merge all the Data together
    statsy[['index','stat']] = statsy['index'].str.split(' , ',expand=True)
    deets[['index','stat']] = deets['index'].str.split(' , ',expand=True)
    deets = deets.rename(columns = {'category':'play_type', 'yards':'yards_gained_play','penalty':'penalty_description'})
    deets = deets[{'index','play_type','result','start_location','end_location', 'direction','yards_gained_play','penalty_description'}]
    statsy_merge = statsy.merge(deets, how = 'inner', on = ['index'])
    merged_df = statsy_merge.merge(hmm, how = 'inner', on = ['index'])
    #creating_more_columns
    merged_df['player_name'] = " "
    merged_df['player_sr_id'] = " "
    merged_df['jersey'] = " "
    merged_df['position'] = " "
    merged_df['players_team'] = " "
    merged_df['penalty_yards'] = " "
    merged_df['penalty_accepted'] = " "
    merged_df['penalty_type'] = " "
    merged_df['penalty_team'] = " "
    merged_df['play_end_yardline'] = " "
    merged_df['play_start_yardline'] = " "
    #Adding some data to the columns from inside the nested json including Jersey Numbers and Player Positions
    for i in range(len(merged_df['player'])):
        try:
            merged_df['player_name'][i] = merged_df['player'][i]['name']
            merged_df['player_sr_id'][i] = merged_df['player'][i]['sr_id']
            merged_df['jersey'][i] = merged_df['player'][i]['jersey']
            merged_df['position'][i] = merged_df['player'][i]['position']
            merged_df['players_team'][i] = merged_df['team_x'][i]['alias']
            merged_df['play_start_yardline'][i] = merged_df['start_location'][i]['yardline']
            merged_df['play_end_yardline'][i] = merged_df['end_location'][i]['yardline']
        except TypeError:
            merged_df['player_name'][i] = "NaN"
            merged_df['player_sr_id'][i] = "NaN"
            merged_df['jersey'][i] = "NaN"
            merged_df['position'][i] = "NaN"
            merged_df['players_team'][i] = "NaN"
            merged_df['play_start_yardline'][i] = "NaN"
            merged_df['play_end_yardline'][i] = "NaN"
            merged_df['penalty_team'][i] = "NaN"
            merged_df['penalty_yards'][i] = "NaN"
            merged_df['penalty_accepted'][i] ="NaN"
            merged_df['penalty_type'][i] ="NaN"
        try:
            merged_df['penalty_type'][i] = merged_df['penalty_description'][i]['description']
            merged_df['penalty_accepted'][i] = merged_df['penalty_description'][i]['result']
            merged_df['penalty_yards'][i] = merged_df['penalty_description'][i]['yards']
            merged_df['penalty_team'][i] = merged_df['penalty_description'][i]['team']['name']
        except KeyError:
            merged_df['penalty_yards'][i] = "NaN"
        except TypeError:
            merged_df['penalty_yards'][i] = "NaN"
            merged_df['penalty_accepted'][i] = "NaN"
            merged_df['penalty_yards'][i] = "NaN"
            merged_df['penalty_team'][i] = "NaN"
    #For some reason we get duplicates on the merge here, so lets create a column using the player associated with the stat type
    #and the play in that game in order to drop the duplicates. 
    merged_df['todropduplicates'] = ' '
    indexs = merged_df['index']
    players_name = merged_df['player_name']
    players_team = merged_df['players_team']
    for i in range(len(merged_df)):
        merged_df['todropduplicates'][i] = f'{indexs[i]}{players_name[i]}{players_team[i]}'
    #drop duplicates
    merged_df = merged_df.drop_duplicates(subset='todropduplicates', keep='first')
    #drop columns we already extracted the data we want from
    merged_df = merged_df.drop(columns={'player','team_x','start_location','end_location','start_situation','end_situation','statistics','details'})
    merged_df = merged_df.reset_index()
    merged_df = merged_df.drop(columns='level_0')
    #add Week, home_team and away_team so that we can merge with nflstatR
    merged_df['week'] = ' ' 
    merged_df['home_team'] = ' ' 
    merged_df['away_team'] = ' ' 
    summary = pd.json_normalize(df_2)
    week = summary['summary.week.sequence'][0]
    home_team = summary['summary.home.alias'][0]
    away_team = summary['summary.away.alias'][0]
    merged_df ['week'] = week
    merged_df ['home_team'] = home_team
    merged_df ['away_team'] = away_team
    #fixing Oakland and Jax
    for i in range(len(merged_df)):
        if (merged_df['home_team'][i] == "OAK"):
            merged_df['home_team'][i] = 'LV'
        elif (merged_df['home_team'][i] == 'JAC'):
            merged_df['home_team'][i] = 'JAX'
        elif(merged_df['away_team'][i] == "OAK"):
            merged_df['away_team'][i] = 'LV'
        elif(merged_df['away_team'][i] == "JAC"):
            merged_df['away_team'][i] = 'JAX'
        else:
            merged_df['home_team'][i] = merged_df['home_team'][i]
            merged_df['away_team'][i] = merged_df['away_team'][i]
    merged_df['for_r'] = f'{home_team}{away_team}{week}'
    return(merged_df)

In [ ]:
#run code below in order to create your massive DF that you will export to csv
master = []
for i in range(len(schedule)):
    print(schedule_for_scraping['Game ID'][i])
    master.append(game_scraper(api_key,schedule_for_scraping['Game ID'][i]))
    time.sleep(2)

In [ ]:
df = pd.concat(master)

In [ ]:
df.to_csv('all_sportradar_pbp.csv')